In [10]:
import tifffile
from matplotlib import pyplot as plt
import numpy as np
from pathlib import Path
import pandas as pd
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from skimage import filters, morphology, measure, segmentation
from tqdm import tqdm
from matplotlib.colors import LinearSegmentedColormap
from scipy.signal import find_peaks
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar

In [11]:
def im_stack_thresh(im_stack): 
    oats = []

    for i in range(len(im_stack)):
        oats.append(filters.threshold_otsu(im_stack[i]))

    return np.mean(oats)

In [12]:
#Get all the folders needed:
folder = '/Volumes/Samsung_T5/Main Data/Analysis Sets/23 Protofigures/Data Sets/2023_05_03_CGD_Accumulation/'
folderpath = Path(folder, )
img_folder = folderpath / 'nuclei'
core_ROI_folder = folderpath / 'nuclei_core_ROI'
seg_folder = folderpath / 'segmented_nuclei'
seg_CORE_folder = folderpath / 'segmented_nuclei_core_ROI'

#Get a list of all the images in each folder: 
img_paths = sorted(img_folder.glob('*.tif'), key= lambda x: x.stem.split('_'))
core_ROI_paths = sorted(core_ROI_folder.glob('*.tif'), key= lambda x: x.stem.split('_'))
seg_paths = sorted(seg_folder.glob('*.tif'), key= lambda x: x.stem.split('_'))
core_ROI_segmentations = sorted(seg_CORE_folder.glob('*.tif'), key= lambda x: x.stem.split('_'))

In [5]:
#For CGD examples: 
#Image 8: img_paths[-2], 18
#Image 11: img_paths[3], 11
#Image 9: img_paths[-1], 4 (zero index)

#Control examples: 
#Image 18: img_paths[10], 7
#Image 5: img_paths[-5], 14
#Image 3: img_paths[-7], 3 (zero index)

#Full Frame: 
#Image 11: img_paths[3], 10


In [5]:
trains_colors = ['#5BCFFA', '#FFFFFF', '#F5ABB9']
train_palette = LinearSegmentedColormap.from_list(colors=trains_colors, name='trains')

In [13]:
export_location = '/Volumes/Samsung_T5/Main Data/Analysis Sets/23 Protofigures/Supplement 5/accumulation_stills/'
#CGD Set: 800-3500, Control: 500-2500
vmin_set = 500
vmax_set = 2500
border_color_core = '#EE362B'
border_color_cells = '#F58963'
border_width = 10


In [14]:
list_position = 3
position_frame = 10

image_name = img_paths[list_position].stem
im_file = tifffile.imread(core_ROI_paths[list_position])[position_frame]
full_frame = tifffile.imread(img_paths[list_position])[position_frame]
seg_file = tifffile.imread(core_ROI_segmentations[list_position])[position_frame]
full_seg = tifffile.imread(seg_paths[list_position])[position_frame]

In [38]:
#Generating stills with boundry of accumulation overlay on nuclear imaging:

#Get the final core centroid first:

threshold = im_stack_thresh(im_file)

frame = full_frame

#Threshold core by otsu thresh for stack, then remove all single cells etc
bin_im = frame > threshold
core_adj = morphology.erosion(bin_im)
core_adj = morphology.remove_small_objects(core_adj, 80)
core_adj = morphology.dilation(core_adj)
boundary_core = measure.find_contours(core_adj)

cell_bin = np.invert(core_adj)*(full_seg > 0)*1.0


#Get the info of the cores detected: 
core_infos = pd.DataFrame(measure.regionprops_table(measure.label(core_adj), properties=['area','centroid']))

#If there are two, take the bigger one as the center one: 
if len(core_infos) > 1: 
    max_idx = core_infos.area.idxmax()
    max_core = core_infos.loc[max_idx]
    core_center = [int(max_core['centroid-1']),int(max_core['centroid-0'])]
else: 
    core_center = [int(core_infos['centroid-1'].values),int(core_infos['centroid-0'].values)]


#Plotting
fig, ax = plt.subplots(figsize=(10,10))
plt.imshow(frame, cmap='Greys_r', vmin=vmin_set,vmax=vmax_set)
plt.axis('off')
    
boundary_cells = measure.find_contours(cell_bin)
for contour in boundary_cells:
    ax.plot(contour[:, 1], contour[:, 0], linewidth=1, color=border_color_cells)
    

for contour in boundary_core:
    ax.plot(contour[:, 1], contour[:, 0], linewidth=2, color=border_color_core)


plt.gca().invert_yaxis()


scalebar = AnchoredSizeBar(ax.transData,
                           200/2.2, '',  'lower right', 
                           pad=1,
                           color='white',
                           frameon=False,
                           size_vertical=10)

ax.add_artist(scalebar)
plt.savefig(export_location + 'full-frame-{}-{}.png'.format(image_name, position_frame), dpi=300, bbox_inches='tight')
plt.close()


In [36]:
#Generating stills with boundry of accumulation overlay on nuclear imaging:

#Get the final core centroid first:

threshold = im_stack_thresh(im_file)

frame = im_file

#Threshold core by otsu thresh for stack, then remove all single cells etc
bin_im = frame > threshold
core_adj = morphology.erosion(bin_im)
core_adj = morphology.remove_small_objects(core_adj, 80)
core_adj = morphology.dilation(core_adj)
boundary_core = measure.find_contours(core_adj)

cell_bin = np.invert(core_adj)*(seg_file > 0)*1.0


#Get the info of the cores detected: 
core_infos = pd.DataFrame(measure.regionprops_table(measure.label(core_adj), properties=['area','centroid']))

#If there are two, take the bigger one as the center one: 
if len(core_infos) > 1: 
    max_idx = core_infos.area.idxmax()
    max_core = core_infos.loc[max_idx]
    core_center = [int(max_core['centroid-1']),int(max_core['centroid-0'])]
else: 
    core_center = [int(core_infos['centroid-1'].values),int(core_infos['centroid-0'].values)]


#Plotting
fig, ax = plt.subplots(figsize=(10,10))
plt.imshow(frame, cmap='Greys_r', vmin=vmin_set,vmax=vmax_set)
plt.axis('off')
cell_label_only = np.invert(core_adj)*seg_file

for i in range(np.max(seg_file)):
    boundary_cells = measure.find_contours(cell_label_only, i)
    for contour in boundary_cells:
        ax.plot(contour[:, 1], contour[:, 0], linewidth=1, color=border_color_cells)
    
for contour in boundary_core:
    ax.plot(contour[:, 1], contour[:, 0], linewidth=4, color=border_color_core)

plt.gca().invert_yaxis()

scalebar = AnchoredSizeBar(ax.transData,
                           200/2.2, '',  'lower right', 
                           pad=1,
                           color='white',
                           frameon=False,
                           size_vertical=10)

ax.add_artist(scalebar)
plt.savefig(export_location + 'crop_ROI-{}-{}.png'.format(image_name, position_frame), dpi=300, bbox_inches='tight')
plt.close()
